In [ ]:
import pandas as pd
import requests
import os
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd

# Set Up the URL 
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
params = {
    "format": "geojson",
    "starttime": "2022-04-01",
    "endtime": "2024-12-31",
    "minmagnitude": 4.5,
}

# Send the Request and Check the Response
response = requests.get(url, params=params)
print(f"Response Status Code: {response.status_code}")
if response.status_code == 200:
    earthquake_data = response.json()
    print("Number of earthquakes:", len(earthquake_data["features"]))
    print("First earthquake:", earthquake_data["features"][0])
elif response.status_code == 400:
    print("Bad Request Error: Check URL and parameters")
    print(response.text)  # Print the full response text for debugging
else:
    print("Error:", response.status_code)

# Ensure the DataFrame df is only created if the response is successful
if response.status_code == 200:
    # Parse the GeoJSON Data and Extract Information
    earthquake_list = []
    for feature in earthquake_data["features"]:
        earthquake = {
            "type": feature["type"],
            "geometry": feature["geometry"],
            "id": feature["id"]
        }
        for key, value in feature["properties"].items():
            earthquake[key] = value
        earthquake_list.append(earthquake)
    df = pd.DataFrame(earthquake_list)
else:
    print("DataFrame 'df' is not defined. Check for issues in the data extraction steps.")

# Extract Latitude, Longitude, and Depth from geometry
if 'df' in locals():
    df['latitude'] = df['geometry'].apply(lambda x: x['coordinates'][1])
    df['longitude'] = df['geometry'].apply(lambda x: x['coordinates'][0])
    df['depth'] = df['geometry'].apply(lambda x: x['coordinates'][2])

    # Convert Timestamp to Datetime
    df['time'] = pd.to_datetime(df['time'], unit='ms')

    # Drop the geometry column if no longer needed
    df.drop(columns=['geometry'], inplace=True)

    # Set 'time' as the index
    df.set_index('time', inplace=True)

    # Display DataFrame information
    print(df.info())
else:
    print("DataFrame 'df' is not defined. Check for issues in the data extraction steps.")

# Convert DataFrame to GeoDataFrame
def create_geodataframe(df):
    geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    return gdf

# Create a GeoDataFrame
gdf = create_geodataframe(df)

# Set Coordinate Reference System (CRS)
gdf.set_crs(epsg=4326, inplace=True)

# Path to the extracted shapefile directory
extract_to_path = r'C:\Users\thoma\Documents\GitHub\USGS\Tectonic_Plates_and_Boundaries'
shapefile_path = os.path.join(extract_to_path, 'Tectonic_Plates_and_Boundaries.shp')

# Load the shapefile using GeoPandas
if os.path.exists(shapefile_path):
    tectonic_plates = gpd.read_file(shapefile_path)
    print(tectonic_plates.head())
else:
    print("Shapefile not found. Please check the path.")

# Ensure the tectonic plates GeoDataFrame has the same CRS as the earthquake GeoDataFrame
tectonic_plates = tectonic_plates.to_crs(gdf.crs)

# Perform spatial join to associate each earthquake with the corresponding tectonic plate
gdf = gpd.sjoin(gdf, tectonic_plates, how='left', predicate='intersects', lsuffix='left', rsuffix='right')

# Rename the column for clarity
gdf.rename(columns={'Name': 'tectonic_plate'}, inplace=True)

# Verify the results of the renaming
print(gdf.columns)
print(gdf.head())

# Ensure 'time' column is retained for feature engineering
gdf.reset_index(inplace=True)

# Calculate time since last significant earthquake within each tectonic plate
gdf['time_since_last'] = gdf.groupby('tectonic_plate')['time'].diff().dt.days

# Define function to count earthquakes within specified days
def count_earthquakes(df, days):
    end_date = df['time'].max()
    start_date = end_date - pd.Timedelta(days=days)
    filtered_df = df[(df['time'] >= start_date) & (df['time'] <= end_date)]
    return filtered_df.groupby('tectonic_plate')['mag'].count()

# Count number of smaller earthquakes in past 1, 3, and 6 months within each tectonic plate
gdf['count_1m'] = gdf.apply(lambda row: count_earthquakes(gdf[gdf['tectonic_plate'] == row['tectonic_plate']], 30).get(row['tectonic_plate'], 0), axis=1)
gdf['count_3m'] = gdf.apply(lambda row: count_earthquakes(gdf[gdf['tectonic_plate'] == row['tectonic_plate']], 90).get(row['tectonic_plate'], 0), axis=1)
gdf['count_6m'] = gdf.apply(lambda row: count_earthquakes(gdf[gdf['tectonic_plate'] == row['tectonic_plate']], 180).get(row['tectonic_plate'], 0), axis=1)

# Rolling averages of seismic activity within each tectonic plate
def rolling_avg_earthquakes(df, days):
    df['rolling_avg'] = df.groupby('tectonic_plate')['mag'].transform(lambda x: x.rolling(window=days, min_periods=1).mean())
    return df['rolling_avg']

gdf['rolling_avg_3m'] = rolling_avg_earthquakes(gdf, 90)

# Fill NA values with appropriate values (e.g., 0 for counts)
gdf.fillna(0, inplace=True)

# Display DataFrame information
print(gdf.info())


Response Status Code: 200
Number of earthquakes: 19601
First earthquake: {'type': 'Feature', 'properties': {'mag': 4.6, 'place': '80 km NW of Kandrian, Papua New Guinea', 'time': 1735602989977, 'updated': 1737523819040, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us6000pgkh', 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us6000pgkh&format=geojson', 'felt': None, 'cdi': None, 'mmi': None, 'alert': None, 'status': 'reviewed', 'tsunami': 0, 'sig': 326, 'net': 'us', 'code': '6000pgkh', 'ids': ',us6000pgkh,', 'sources': ',us,', 'types': ',origin,phase-data,', 'nst': 47, 'dmin': 3.367, 'rms': 0.44, 'gap': 109, 'magType': 'mb', 'type': 'earthquake', 'title': 'M 4.6 - 80 km NW of Kandrian, Papua New Guinea'}, 'geometry': {'type': 'Point', 'coordinates': [148.9729, -5.7603, 127.013]}, 'id': 'us6000pgkh'}
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 19601 entries, 2024-12-30 23:56:29.977000 to 2022-04-01 01:08:14.815000
Data columns (total 

C:\Users\thoma\AppData\Local\Temp\ipykernel_9828\3224999938.py:129: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdf.fillna(0, inplace=True)


# Model Data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier

In [14]:
# Define features and target variable
features = ['time_since_last', 'count_1m', 'count_3m', 'count_6m', 'rolling_avg_3m', 'latitude', 'longitude']
threshold_magnitude = 6.0  # Define your threshold magnitude
X = gdf[features]
y = (gdf['mag'] >= threshold_magnitude).astype(int)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Train the model
log_reg_model = LogisticRegression(random_state=42)
log_reg_model.fit(X_train, y_train)

# Make predictions
y_pred_log_reg = log_reg_model.predict(X_test)
y_prob_log_reg = log_reg_model.predict_proba(X_test)[:, 1]

# Evaluate the Logistic Regression model
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
roc_auc_log_reg = roc_auc_score(y_test, y_prob_log_reg)

print("Logistic Regression Accuracy:", round(accuracy_log_reg, 3))
print("Logistic Regression ROC-AUC Score:", round(roc_auc_log_reg, 3))

Logistic Regression Accuracy: 0.981
Logistic Regression ROC-AUC Score: 0.454


In [16]:
# Train the model
decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(X_train, y_train)

# Make predictions
y_pred_dt = decision_tree_model.predict(X_test)
y_prob_dt = decision_tree_model.predict_proba(X_test)[:, 1]

# Evaluate the Decision Tree model
accuracy_dt = accuracy_score(y_test, y_pred_dt)
roc_auc_dt = roc_auc_score(y_test, y_prob_dt)

print("Decision Tree Accuracy:", round(accuracy_dt, 3))
print("Decision Tree ROC-AUC Score:", round(roc_auc_dt, 3))



Decision Tree Accuracy: 0.967
Decision Tree ROC-AUC Score: 0.506


In [ ]:
# Train the model
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# Make predictions
y_pred_gb = gb_model.predict(X_test)
y_prob_gb = gb_model.predict_proba(X_test)[:, 1]

# Evaluate the Gradient Boosting model
accuracy_gb = accuracy_score(y_test, y_pred_gb)
roc_auc_gb = roc_auc_score(y_test, y_prob_gb)

print("Gradient Boosting Accuracy:", round(accuracy_gb, 3))
print("Gradient Boosting ROC-AUC Score:", round(roc_auc_gb, 3))


Gradient Boosting Accuracy: 0.973
Gradient Boosting ROC-AUC Score: 0.531


In [19]:
# Train the CatBoost model
catboost_model = CatBoostClassifier(iterations=100, random_state=42, verbose=0)
catboost_model.fit(X_train, y_train)

# Make predictions
y_pred_cb = catboost_model.predict(X_test)
y_prob_cb = catboost_model.predict_proba(X_test)[:, 1]

# Evaluate the CatBoost model
accuracy_cb = accuracy_score(y_test, y_pred_cb)
roc_auc_cb = roc_auc_score(y_test, y_prob_cb)

print("CatBoost Accuracy:", round(accuracy_cb, 3))
print("CatBoost ROC-AUC Score:", round(roc_auc_cb, 3))


CatBoost Accuracy: 0.981
CatBoost ROC-AUC Score: 0.587


In [20]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

# Define features and target variable
features = ['time_since_last', 'count_1m', 'count_3m', 'count_6m', 'rolling_avg_3m', 'latitude', 'longitude']
X = gdf[features]
y = (gdf['mag'] >= 6.0).astype(int)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the LightGBM model
lgbm_model = LGBMClassifier(n_estimators=100, random_state=42)
lgbm_model.fit(X_train, y_train)

# Make predictions
y_pred_lgbm = lgbm_model.predict(X_test)
y_prob_lgbm = lgbm_model.predict_proba(X_test)[:, 1]

# Evaluate the LightGBM model
accuracy_lgbm = accuracy_score(y_test, y_pred_lgbm)
roc_auc_lgbm = roc_auc_score(y_test, y_prob_lgbm)

print("LightGBM Accuracy:", round(accuracy_lgbm, 3))
print("LightGBM ROC-AUC Score:", round(roc_auc_lgbm, 3))


[LightGBM] [Info] Number of positive: 255, number of negative: 15425
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000111 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 15680, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.016263 -> initscore=-4.102481
[LightGBM] [Info] Start training from score -4.102481
LightGBM Accuracy: 0.981
LightGBM ROC-AUC Score: 0.578
